# Ensemble Learning

## Initial Imports

In [47]:
import warnings
warnings.filterwarnings('ignore')

In [48]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [49]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.model_selection import train_test_split

## Read the CSV and Perform Basic Data Cleaning

In [73]:


columns = [
    "loan_amnt", "int_rate", "installment","home_ownership",
    "annual_inc", "verification_status", "issue_d",
    "loan_status", "pymnt_plan", "dti", "delinq_2yrs",
    "inq_last_6mths", "open_acc", "pub_rec", "revol_bal",
    "total_acc", "initial_list_status", "out_prncp",
    "out_prncp_inv", "total_pymnt", "total_pymnt_inv", 
    "total_rec_prncp", "total_rec_int", "total_rec_late_fee", 
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt",
    "next_pymnt_d", "collections_12_mths_ex_med", "policy_code",
    "application_type", "acc_now_delinq", "tot_coll_amt", "tot_cur_bal",
    "open_acc_6m", "open_act_il", "open_il_12m", "open_il_24m", "mths_since_rcnt_il",
    "total_bal_il", "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl", "inq_last_12m", "acc_open_past_24mths",
    "avg_cur_bal", "bc_open_to_buy", "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc", "mths_since_recent_bc", 
    "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl", "num_actv_rev_tl", "num_bc_sats", 
    "num_bc_tl", "num_il_tl", "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0", "num_sats", 
    "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m", "num_tl_op_past_12m", "pct_tl_nvr_dlq",
    "percent_bc_gt_75", "pub_rec_bankruptcies", "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit", 
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"

]

target = ["loan_status"]

In [51]:
pwd

'C:\\Users\\rmwil\\OneDrive\\desktop\\Classwork\\11---Machine-Learning\\Instructions\\Starter_Code'

In [52]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, index_col="issue_d", parse_dates=True, infer_datetime_format=True)




df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
issue_d,,,,,,,,,,,,,,,,,,,,,
2019-03-01,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,low_risk,n,27.24,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
2019-03-01,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,low_risk,n,20.23,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2019-03-01,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,low_risk,n,24.26,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
2019-03-01,10000.0,0.1640,353.55,RENT,92000.0,Verified,low_risk,n,31.44,0.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
2019-03-01,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,low_risk,n,18.76,0.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [54]:
binary_encoded_df = pd.get_dummies(df, columns =['home_ownership', 'verification_status', 'pymnt_plan','initial_list_status','next_pymnt_d','application_type','hardship_flag','debt_settlement_flag'])
binary_encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 68817 entries, 2019-03-01 to 2019-01-01
Data columns (total 93 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   loan_amnt                            68817 non-null  float64
 1   int_rate                             68817 non-null  float64
 2   installment                          68817 non-null  float64
 3   annual_inc                           68817 non-null  float64
 4   loan_status                          68817 non-null  object 
 5   dti                                  68817 non-null  float64
 6   delinq_2yrs                          68817 non-null  float64
 7   inq_last_6mths                       68817 non-null  float64
 8   open_acc                             68817 non-null  float64
 9   pub_rec                              68817 non-null  float64
 10  revol_bal                            68817 non-null  float64
 11  total_acc  

## Split the Data into Training and Testing

In [55]:
# Create our features
X = df.drop(columns='loan_status')
X = pd.get_dummies(X)

# Create our target
target = ["loan_status"]
y = df.loc[:, target].copy()


In [56]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.148001,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.355104,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [57]:
# Check the balance of our target values
df['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [58]:
# Split the X and y into X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

X_train.shape

(51612, 92)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [59]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [60]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = data_scaler.fit(X_train)
X_scaler

StandardScaler()

In [61]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [64]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = RandomForestClassifier(n_estimators = 100, random_state = 1)
rf_model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=1)

In [65]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.6490747791193783

In [66]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   26,    61],
       [   12, 17106]], dtype=int64)

In [67]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.68      0.30      1.00      0.42      0.55      0.28        87
   low_risk       1.00      1.00      0.30      1.00      0.55      0.32     17118

avg / total       0.99      1.00      0.30      0.99      0.55      0.32     17205



In [68]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.07797760885159455, 'total_rec_int'),
 (0.06369988514786096, 'last_pymnt_amnt'),
 (0.06304033390219775, 'total_pymnt'),
 (0.06261650245037574, 'total_pymnt_inv'),
 (0.059319306893811444, 'total_rec_prncp'),
 (0.017727738854647707, 'out_prncp_inv'),
 (0.01658935845661797, 'installment'),
 (0.016501994744745365, 'mo_sin_old_il_acct'),
 (0.016346941740589985, 'revol_bal'),
 (0.016333393009448152, 'total_rec_late_fee'),
 (0.016092033841037013, 'mo_sin_old_rev_tl_op'),
 (0.015939154020555072, 'tot_cur_bal'),
 (0.015904781861977316, 'dti'),
 (0.015554847774414461, 'bc_open_to_buy'),
 (0.015426292876281117, 'tot_hi_cred_lim'),
 (0.015334199644022101, 'total_bal_ex_mort'),
 (0.01519723940739414, 'max_bal_bc'),
 (0.01482329147796266, 'out_prncp'),
 (0.01480718072338848, 'total_bal_il'),
 (0.014706955077939161, 'avg_cur_bal'),
 (0.014619825631701697, 'annual_inc'),
 (0.014458742460282841, 'bc_util'),
 (0.013939715097261269, 'total_bc_limit'),
 (0.013611027459292081, 'total_rev_hi_lim'),
 (0.0

### Easy Ensemble Classifier

In [69]:
# Train the Classifier
ee_model = EasyEnsembleClassifier(n_estimators = 500, random_state = 1)
ee_model.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=500, random_state=1)

In [70]:
# Calculated the balanced accuracy score
y_pred = ee_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.75222089270822

In [71]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   62,    25],
       [ 3564, 13554]], dtype=int64)

In [72]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.71      0.79      0.03      0.75      0.56        87
   low_risk       1.00      0.79      0.71      0.88      0.75      0.57     17118

avg / total       0.99      0.79      0.71      0.88      0.75      0.57     17205



In [ ]:
### Final Questions

1. Which model had the best balanced accuracy score?

    ## The Easy Ensemble has the best balanced accuracy score.

2. Which model had the best recall score?

    The Balanced Random Forrest Classifer has the best recall score.

3. Which model had the best geometric mean score?

    ## The Easy Ensemble has the best balanced accuracy score

4. What are the top three features?

    ## Precision, Recall & Fl-Score